In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):
    def __init__(self,ntoken,nhead,ninp,nlayers,ndropout=0.5):
        super(TransformerModel,self).__init__()
        from torch.nn import TransformerEncoder,TransformerEncoderLayer
        self.model_type='Transformer'
        self.src_mask=None
        self.pos_encoder=PositionalEncoding(ninp,dropout)
        encoder_layers=TransformerEncoderLayer(ninp,nhead,nhid,ndropout)
        self.transformer_encoder=TransformerEncoder(encoder_layers,nlayers)
        self.encoder=nn.Embedding(ntoken,ninp)
        self.ninp=ninp
        self.decoder=nn.Linear(ninp,ntoken)
        
        self.init_weights()
        
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
        
    def init_weights(self):
        initrange=0.1
        self.encoder.weight.data.uniform_(-initrange,initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange,initrange)
            
    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0)!=len(src):
            device=src.device
            mask=self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask=mask
        
        src=self.encoder(src)*math.sqrt(self.ninp)
        src=self.pos_encoder(src)
        output=self.transformer_encoder(src,self.src_mask)
        output=self.decoder(output)
        return F.log_softmax(output,dim=-1)

In [2]:
class PositionalEncoding(nn.Module):
    
    def __init__(self,d_model,dropout=0.1,max_len=5000):
        super(PositionalEncoding,self).__init__()
        self.dropout=nn.Dropout(p=dropout)
        pe=torch.zeros(max_len,d_model)
        position=torch.arange(0,max_len,dtype=torch.float).unsqueeze(1)
        div_term=torch.exp(torch.arange(0,d_model,2).float()*(-math.log(10000.0)/d_model))
        pe[:, 0::2]=torch.sin(position*div_term)
        pe[:, 1::2]=torch.cos(position*div_term)
        pe=pe.unsqueeze(0).transpose(0,1)
        self.register_buffer('pe',pe)
        
    def forward(self,x):
        x=x+self.pe[:x.size(0),:]
        return self.dropout(x)

In [3]:
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT=torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                          init_token='<sos>',
                          eos_token='<eos>',
                          lower=True)
train_txt,val_txt,test_txt=torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data,bsz):
    data=TEXT.numericalize([data.examples[0].text])
    nbatch=data.size(0)//bsz
    #Trim off any element that wouldnt fit
    data=data.narrow(0,0,nbatch*bsz)
    data=data.view(bsz,-1).t().contiguous()
    return data.to(device)

batch_size=20
eval_batch_size=10
train_data=batchify(train_txt,batch_size)
val_data=batchify(val_txt,eval_batch_size)
test_data=batchify(test_txt,eval_batch_size)

In [4]:
from platform import python_version

print(python_version())

3.5.2


In [5]:
bptt=35
def get_batch(source,i):
    seq_len=min(bptt,len(source)-1-i)
    data=source[i:i+seq_len]
    target=source[i+1:i+seq_len+1].view(-1)
    #print("DATA-{}-{}-:".format(i,type(data)))
    return(data,target)

In [6]:
ntokens=len(TEXT.vocab.stoi)#size of vocabulary
emsize=200 # embedding dimension
nhid=200 # number of hidden layer
nlayers=2# the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead=2 # number of heads in multidimension attention
dropout=0.2
model=TransformerModel(ntokens,emsize,nhid,nlayers,dropout).to(device)

In [19]:
criterion=nn.CrossEntropyLoss()
lr=0.5
optimizer=torch.optim.SGD(model.parameters(),lr=lr)
scheduler=torch.optim.lr_scheduler.StepLR(optimizer,1.0,gamma=0.95)

import time
def train():
    model.train()
    total_loss=0.
    start_time=time.time()
    ntokens=len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0,train_data.size(0)-1,bptt)):# From where are we getting train data values
        data,targets=get_batch(train_data,i)
        optimizer.zero_grad()
        output=model(data)
        loss=criterion(output.view(-1,ntokens),targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),0.5)
        optimizer.step()
        
        total_loss +=loss.item()
        log_interval=200
        if batch%log_interval==0 and batch>0:
            cur_loss=total_loss/log_interval
            elapsed=time.time()-start_time
            print('| epoch {:3d} | {:5d}/{:3d} batches'
                  'lr {:02.2f} | ms/batch {:5.2f}'
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                   epoch,batch,len(train_data)//bptt,scheduler.get_lr()[0],
                   elapsed*1000/log_interval,cur_loss,math.exp(cur_loss)))
            total_loss=0
            start_time=time.time()
            
    
def evaluate(eval_model,data_source):
    eval_model.eval()
    total_loss=0
    ntokens=len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0,data_source.size(0)-1,bptt):
            data,targets=get_batch(data_source,i)
            output=eval_model(data)
            output_flat=output.view(-1,ntokens)
            total_loss+=len(data)*criterion(output_flat,targets).item()     
    return total_loss/(len(data_source)-1)

In [20]:
best_val_loss=float("inf")
epochs=3
best_model=None

for epoch in range(1,epochs+1):
    epoch_start_time=time.time()
    train()
    val_loss=evaluate(model,val_data)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | valid ppl {:8.2f}'.format(epoch,(time.time()-epoch_start_time),val_loss,math.exp(val_loss)))
    print('-'*89)
    
    if val_loss<best_val_loss:
        best_val_loss=val_loss
        best_model=model
    
    scheduler.step()

| epoch   1 |   200/2981 batcheslr 0.50 | ms/batch 24.85loss  5.03 | ppl   153.05
| epoch   1 |   400/2981 batcheslr 0.50 | ms/batch 24.98loss  5.08 | ppl   160.23
| epoch   1 |   600/2981 batcheslr 0.50 | ms/batch 25.31loss  4.88 | ppl   132.27
| epoch   1 |   800/2981 batcheslr 0.50 | ms/batch 24.76loss  4.98 | ppl   144.99
| epoch   1 |  1000/2981 batcheslr 0.50 | ms/batch 24.51loss  4.93 | ppl   137.81
| epoch   1 |  1200/2981 batcheslr 0.50 | ms/batch 24.43loss  4.97 | ppl   144.36
| epoch   1 |  1400/2981 batcheslr 0.50 | ms/batch 24.45loss  5.00 | ppl   149.10
| epoch   1 |  1600/2981 batcheslr 0.50 | ms/batch 24.47loss  5.06 | ppl   158.25
| epoch   1 |  1800/2981 batcheslr 0.50 | ms/batch 24.37loss  4.98 | ppl   145.76
| epoch   1 |  2000/2981 batcheslr 0.50 | ms/batch 24.57loss  5.02 | ppl   150.87
| epoch   1 |  2200/2981 batcheslr 0.50 | ms/batch 24.55loss  4.88 | ppl   131.50
| epoch   1 |  2400/2981 batcheslr 0.50 | ms/batch 24.90loss  4.97 | ppl   144.26
| epoch   1 |  2

In [22]:
test_loss=evaluate(best_model,test_data)
print('='*89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(test_loss,math.exp(test_loss)))
print('='*89)

| End of training | test loss  5.08 | test ppl   160.89
